# GO GO GO

In [3]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from PIL import Image
import timm
import torch
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
from sklearn.metrics import log_loss

import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score
from sklearn.datasets import make_classification
from xgboost import XGBClassifier

# Loading the model

In [37]:
base_model = timm.create_model(
    'resnetv2_152x2_bit.goog_teacher_in21k_ft_in1k_384',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model.to(device)

for param in base_model.parameters():
    param.requires_grad = False

base_model = base_model.eval()

model.safetensors:   0%|          | 0.00/945M [00:00<?, ?B/s]

# Getting new features

In [39]:
train_df = pd.read_csv('/kaggle/input/dataset/train.csv')

In [40]:
X_features = []

data_config = timm.data.resolve_model_data_config(base_model)
transforms = timm.data.create_transform(**data_config, is_training=False)

for img_path in tqdm(train_df['id']):
    
    if len(img_path.split('.')) == 1:
        img_path = f'/kaggle/input/dataset/images/{img_path}.png'
    else:
        img_path = f'/kaggle/input/dataset/images/{img_path}'
        
    img = Image.open(img_path)
    img = img.convert('RGB')
    
    output = base_model.forward_head(output, pre_logits=True)
    result = output.flatten().detach().cpu().numpy()  # переносим данные между CPU и GPU

    X_features.append(result)

X_features = np.array(X_features)

100%|██████████| 1012/1012 [02:28<00:00,  6.83it/s]


In [41]:
X_features.shape

(1012, 4096)

# Boosting training

In [42]:
#Создание объекта модели XGBoost
model = XGBClassifier(n_estimators=1000,
                      learning_rate=0.1,
                      objective='binary:logistic',
                      reg_lambda=10,
                      random_state=42,
                      max_depth=10,
                      subsample=0.8,
                      colsample_bytree=0.8,
                      device='cuda')

# Создание объекта KFold кросс-валидации
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

# Выполнение кросс-валидации и вывод средней оценки log loss
results = cross_val_score(model,  X_features, train_df['target'], cv=kfold, scoring='neg_log_loss')
print(f'Средний log loss модели на кросс-валидации: {results.mean()}')

Средний log loss модели на кросс-валидации: -0.05897274590309281


In [43]:
model = XGBClassifier(n_estimators=1000,
                      learning_rate=0.1,
                      objective='binary:logistic',
                      reg_lambda=10,
                      random_state=42,
                      max_depth=10,
                      subsample=0.8,
                      colsample_bytree=0.8,
                      device='cuda')

model.fit(X_features, train_df['target'])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

# Validation

In [44]:
val_df = pd.read_csv('/kaggle/input/dataset/val.csv')

In [ ]:
# Извлечение признаков и предсказание вероятностей для каждого изображения
X_val_features = []

data_config = timm.data.resolve_model_data_config(base_model)
transforms = timm.data.create_transform(**data_config, is_training=False)

for img_path in tqdm(test_df['id']):
    
    if len(img_path.split('.')) == 1:
        img_path = f'/kaggle/input/dataset/images/{img_path}.png'
    else:
        img_path = f'/kaggle/input/dataset/images/{img_path}'
    
    img = Image.open(img_path)
    img = img.convert('RGB')

    output = base_model.forward_head(output, pre_logits=True)
    result = output.flatten().detach().cpu().numpy() 

    X_val_features.append(result)

X_val_features = np.array(X_val_features)

In [ ]:
# Получение вероятностей принадлежности к классам для тестовых данных
predictions_proba = model.predict_proba(X_val_features)
val_df['pred_target'] = predictions_proba[:, 1]

In [47]:
score = log_loss(val_df.target, val_df.pred_target)
score

0.04835334824863326

# Submit

In [ ]:
test_df = pd.read_csv('/kaggle/input/dataset/test.csv')

In [45]:
# Извлечение признаков и предсказание вероятностей для каждого изображения
X_test_features = []

data_config = timm.data.resolve_model_data_config(base_model)
transforms = timm.data.create_transform(**data_config, is_training=False)

for img_path in tqdm(test_df['id']):
    
    if len(img_path.split('.')) == 1:
        img_path = f'/kaggle/input/dataset/images/{img_path}.png'
    else:
        img_path = f'/kaggle/input/dataset/images/{img_path}'
    
    img = Image.open(img_path)
    img = img.convert('RGB')

    output = base_model.forward_head(output, pre_logits=True)
    result = output.flatten().detach().cpu().numpy()  # переносим данные между CPU и GPU

    X_test_features.append(result)

X_test_features = np.array(X_test_features)

100%|██████████| 506/506 [01:13<00:00,  6.88it/s]


In [46]:
# Получение вероятностей принадлежности к классам для тестовых данных
predictions_proba = model.predict_proba(X_test_features)

submission_df = test_df.copy()
submission_df['target'] = predictions_proba[:, 1]
submission_df.head()

,id,target
0,Qt1fGUB0Vz.jpeg,0.000881
1,j4Rhioq7R3.jpeg,0.005183
2,rD0hgFHJUZ.jpeg,0.012474
3,aY5z1EJsJ6.jpeg,0.998880
4,qZ3IoxD2TE.jpeg,0.024619


In [48]:
submission_df.to_csv('submission_resnet152.csv', index=None)